### HOUSE PRICING 1 : EXTRACTION 

Import the necessary libraries

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import boto3
import numpy as np

Set the parameters to start our AWS client

In [129]:
params = {
    "region": "region",
    "bucket": "bucket", 
    "key":"key",
    "folder": "folder",
    "secretkey":"secretkey"}

Start the s3 client, we will use it later

In [130]:
s3 = boto3.client(
        's3', 
        region_name = params['region'],
        aws_access_key_id = params['key'],
        aws_secret_access_key = params['secretkey']
    )

Create a empty dataframe

In [131]:
real_estate = pd.DataFrame(columns=['Localizacion','Estado' ,'Precio', 'Cuartos', 'Baños', 'Cocheras', 
                                    'Area_total'])

I decided to scrape just the 10 richest states in Mexiko

In [132]:
Estados = ['estado-de-mexico','veracruz','chihuahua', 'coahuila', 'baja-california',
           'sonora', 'jalisco', 'guanajuato','distrito-federal', 'nuevo-leon']

I define a function to avoid errors with null values

In [133]:
def get_item_by_class(result_reg,localizador, clase):
    '''
    This funtions gets the text from determinated instances using the class
    '''
    lista = list()
    for i in result_reg:
        try:
            item = i.find(localizador, {'class': clase}).get_text()
            lista.append(item)
        except:
            lista.append('Null')
    return lista

Scrape the information. I use 50 pages per state and every page give us 23 new rows

In [134]:
for j in Estados:
    print('Estamos en la iteracion: ', str(j))
    for i in range(51):
        website = requests.get('https://www.vivanuncios.com.mx/s-venta-inmuebles/'+j+'/page-'+str(i)+'/v1c1097l1014p'+str(i))
        soup = BeautifulSoup(website.content, 'html.parser')

        result_reg = soup.find_all('div', {'class':'tileV2 REAdTileV2 regular listView'})

        Localizacion = get_item_by_class(result_reg,'div', 'tile-location one-liner')
        
        Estado = [j for i in result_reg]
                    
        Cocheras = get_item_by_class(result_reg,'div', 'chiplets-inline-block car-parking')
                    
        Cuartos = get_item_by_class(result_reg,'div', 'chiplets-inline-block re-bedroom')
                    
        Baños = get_item_by_class(result_reg,'div', 'chiplets-inline-block re-bathroom')
                    
        Area_total = get_item_by_class(result_reg,'div', 'chiplets-inline-block surface-area')
                    
        Precio = get_item_by_class(result_reg,'span', 'ad-price')
        Precio = [i.strip() for i in Precio]


        for k in range(len(Localizacion)):
            real_estate = real_estate.append({'Localizacion' : Localizacion[k],  'Estado' : Estado[k], 
                                              'Precio': Precio[k], 'Cuartos': Cuartos[k], 'Baños' : Baños[k], 
                                              'Cocheras': Cocheras[k], 'Area_total': Area_total[k]},
                                            ignore_index=True)

Estamos en la iteracion:  estado-de-mexico
Estamos en la iteracion:  veracruz
Estamos en la iteracion:  chihuahua
Estamos en la iteracion:  coahuila
Estamos en la iteracion:  baja-california
Estamos en la iteracion:  sonora
Estamos en la iteracion:  jalisco
Estamos en la iteracion:  guanajuato
Estamos en la iteracion:  distrito-federal
Estamos en la iteracion:  nuevo-leon


In [135]:
real_estate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11419 entries, 0 to 11418
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Localizacion  11419 non-null  object
 1   Estado        11419 non-null  object
 2   Precio        11419 non-null  object
 3   Cuartos       11419 non-null  object
 4   Baños         11419 non-null  object
 5   Cocheras      11419 non-null  object
 6   Area_total    11419 non-null  object
dtypes: object(7)
memory usage: 624.6+ KB


In [136]:
real_estate.head(10)

,Localizacion,Estado,Precio,Cuartos,Baños,Cocheras,Area_total
0,Campestre del Lago,estado-de-mexico,"$5,500,000",3,3,4+,300 m²
1,Méx.,estado-de-mexico,"$1,450,000",3,1,1,90 m²
2,Delegación San Mateo Otzacatipan,estado-de-mexico,"$1,290,000",3,1,Null,96 m²
3,San Bartolo,estado-de-mexico,"$650,000",2,1,1,60 m²
4,Hacienda Ojo de Agua,estado-de-mexico,"$5,500,000",4,4+,3,610 m²
5,Zumpango,estado-de-mexico,"$495,000",2,1,1,Null
6,Fracc. Paseos del Lago,estado-de-mexico,"$650,000",2,1,1,60 m²
7,Las Fuentes Ecatepec,estado-de-mexico,"$2,300,000",2,2,2,149 m²
8,Carlos Hank Gonzalez,estado-de-mexico,"$2,950,000",5,2,2,210 m²
9,Covadonga,estado-de-mexico,"$1,200,000",3,3,2,Null


I save the dataframe to csv

In [137]:
real_estate.to_csv('real_state.csv', index= False)

Upload the file to s3 in the bucket and folder defined in the parameters

In [145]:
s3.upload_file('real_state.csv', params['bucket'], params['folder'] +'real_state.csv')       
